# Cold Start 해결방안

## Test에 새로 등장한 유저가 읽은 책(A)
- **해당 책(A)에 대해 평가한 다른 유저**가 존재하는 경우: 평균 평점
- 해당 책(A)을 읽은 유저가 없는 경우
    - 해당 책을 쓴 **작가(book_author)가 쓴 다른 책**의 평균 평점
- 해당 책을 쓴 작가가 쓴 다른 책이 없는 경우
    - user 정보 활용: **나이(age) & 지역(city)이 같은 유저**의 평균 평점
    

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

from tqdm import tqdm

In [2]:
path='/opt/ml/data/'

users = pd.read_csv(path+'users_preprocessed.csv')
books = pd.read_csv(path+'books_preprocessed.csv')
train_ratings = pd.read_csv(path+'train_preprocessed.csv')
test_ratings = pd.read_csv(path+'test_ratings.csv')


print('users shape: ', users.shape)
print('books shape: ', books.shape)
print('train_ratings shape: ', train_ratings.shape)
print('test_ratings shape: ', test_ratings.shape)

users shape:  (68092, 5)
books shape:  (149570, 7)
train_ratings shape:  (306795, 13)
test_ratings shape:  (76699, 3)


In [3]:
# test에만 있는 NEW_USERS
train_user_id = set(train_ratings['user_id'].unique()) # train에 있는 USER_ID
train_isbn = set(train_ratings['isbn'].unique())

test_user_id = set(test_ratings['user_id'].unique()) # test에 있는 USER_ID
test_isbn = set(test_ratings['isbn'].unique())

total_user_id = set(users['user_id'].unique())
total_isbn = set(books['isbn'].unique())

new_user_id = list(test_user_id - train_user_id) # test에만 있는 USER_ID
new_isbn_id = list(test_isbn - train_isbn) # test에만 있는 ISBN

- 해당 책(A)을 읽은 유저가 없는 경우
    - 해당 책을 쓴 **작가(book_author)가 쓴 다른 책들**이 받은 평균 평점의 가중평균

In [4]:
len(new_user_id) # new_user_id의 demographic info는 users에 다 있음

8266

1. **해당 책(A)에 대해 평가한 다른 유저**가 존재하는 경우: 평균 평점

In [5]:
test_data = test_ratings.merge(users, on='user_id', how='left').merge(books[['isbn', 'category_high', 'pnumber', 'language', 'book_author']], on='isbn',how='left')

In [6]:
# test_data에서 new_user_id에 해당하는 행들만 추출(train data에 한번도 나타나지 않았던 user)
filtered_test_users = test_data[test_data['user_id'].isin(new_user_id)].copy()

In [7]:
filtered_test_users['user_id'].nunique() # test_ratings에만 존재하는 user

8266

In [8]:
# train_ratings 데이터프레임에서 isbn을 그룹으로 묶고, 각 그룹에 대한 평균을 계산하여 ratings_by_isbn이라는 새로운 데이터프레임을 생성
ratings_by_isbn = train_ratings.groupby('isbn')['rating'].mean()

# ratings_by_isbn을 이용하여 추출된 행들의 rating 값을 채우기
filtered_test_users.loc[:, 'rating'] = filtered_test_users['isbn'].map(ratings_by_isbn)

In [9]:
# rule-1 적용 후 null값인 rating
filtered_test_users['rating'].isna().sum()

1734

In [10]:
filtered_test_users.shape

(8677, 11)

2. 해당 책(A)을 읽은 유저가 없는 경우
    - 해당 책을 쓴 **작가(book_author)가 쓴 다른 책**의 평균 평점

In [11]:
# 해당 책을 읽은 유저가 없다.
# test_ratings['other_users_exist'] = test_ratings['isbn'].isin(train_isbn)
# 없는 경우, other_users_exist: False

In [12]:
# mean: 받은 평점, count: 평가한 사용자 수
# to_fill: 해당 책을 쓴 작가가 다른 책에 대해 받은 평점들
to_fill = train_ratings.groupby(['book_author','isbn']).agg({'rating':['mean','count']})['rating'].reset_index()
rating_counts_per_user = to_fill.groupby('book_author').agg({'count':'sum'}).reset_index()
rating_counts_per_user.rename(columns = {'count': 'count_sum'}, inplace = True) # book_author가 받은 평가의 개수

In [13]:
# 다른 책들이 받은 평균 평점 -> 가중 평균
# book_author별로 mean은 sum하고
# 해당 유저가 받은 count sum하고 해당 책에 
to_fill = to_fill.merge(rating_counts_per_user,on='book_author',how='inner')

In [14]:
to_fill['ratio'] = to_fill['count'] / to_fill['count_sum'] 
to_fill['final_ratings'] = to_fill['mean'] * to_fill['ratio']
ratings_per_author = to_fill.groupby('book_author').agg({'final_ratings':sum}).reset_index()
ratings_per_author.rename(columns = {'final_ratings': 'predicted_ratings'}, inplace = True)

to_fill = to_fill.merge(ratings_per_author,on='book_author',how='left')

In [15]:
to_fill = to_fill[['book_author','predicted_ratings']]
to_fill = to_fill.drop_duplicates()

In [16]:
filtered_test_users = filtered_test_users.merge(to_fill,on='book_author',how='left')
print(filtered_test_users.shape)

(8677, 12)


In [17]:
filtered_test_users.loc[filtered_test_users[filtered_test_users['rating'].isnull()].index, 'rating'] = filtered_test_users[filtered_test_users['rating'].isnull()]['predicted_ratings']

In [18]:
filtered_test_users = filtered_test_users.drop('predicted_ratings', axis=1)

In [19]:
filtered_test_users.shape

(8677, 11)

In [20]:
filtered_test_users.isnull().sum()

user_id               0
isbn                  0
rating              725
location_city         0
location_state        0
location_country      0
binning_age           0
category_high         0
pnumber               0
language              0
book_author           0
dtype: int64

3. 해당 책을 쓴 작가가 쓴 다른 책이 없는 경우
    - user 정보 활용: **나이(age) & 지역(city)이 같은 유저**의 평균 평점

In [21]:
age_city_mean_rating = filtered_test_users.groupby(['binning_age','location_city']).agg({'rating':'mean'}).reset_index()

In [22]:
age_city_mean_rating.rename(columns = {'rating': 'predicted_rating'}, inplace = True)

In [23]:
filtered_test_users = filtered_test_users.merge(age_city_mean_rating,on=['binning_age','location_city'],how='left')

In [24]:
filtered_test_users.loc[filtered_test_users[filtered_test_users['rating'].isnull()].index, 'rating'] = filtered_test_users[filtered_test_users['rating'].isnull()]['predicted_rating']

In [25]:
filtered_test_users = filtered_test_users.drop('predicted_rating',axis=1)

In [26]:
filtered_test_users.isna().sum()

user_id               0
isbn                  0
rating              325
location_city         0
location_state        0
location_country      0
binning_age           0
category_high         0
pnumber               0
language              0
book_author           0
dtype: int64

In [27]:
filtered_test_users.to_csv('filtered_test_users.csv')

In [28]:
filtered_test_users.loc[filtered_test_users[filtered_test_users['rating'].isnull()].index, 'rating'] = train_ratings['rating'].mean()

In [29]:
filtered_test_users.isnull().sum()

user_id             0
isbn                0
rating              0
location_city       0
location_state      0
location_country    0
binning_age         0
category_high       0
pnumber             0
language            0
book_author         0
dtype: int64